In [2]:
%reload_ext autoreload
%autoreload 2

In [631]:
import sys
# just in case reload_ext does not work
# Remove the modules if they exist in sys.modules
for mod in ["typst_importer.typst_to_svg", "typst_importer.curve_utils", "typst_importer.svg_preprocessing"]:
    if mod in sys.modules:
        print(f"Removing {mod} from sys.modules")
        del sys.modules[mod]

# Now Python 'forgets' those imports. Reimport them:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

Removing typst_importer.typst_to_svg from sys.modules
Removing typst_importer.curve_utils from sys.modules
Removing typst_importer.svg_preprocessing from sys.modules


In [3]:
import bpy
import sys
import tempfile
import typst
from pathlib import Path

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))

from typst_importer.notebook_utils import display_svg

#uv pip install lxml
#uv pip install numpy==1.26.4
# so that code highlighting works

temp_dir = Path(tempfile.gettempdir())
#temp_dir = Path.cwd()
svg_file = temp_dir / "step1.svg"

Info: Cleared System Clipboard


# Select Typst File

In [4]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

$ a= b/c $  
"""
typst_file = temp_dir / "step1.typ"
typst_file.write_text(file_content)

99

In [634]:
typst_path = Path.home() / "projects/blender_typst_importer/docs/"
typst_file = typst_path / "matrix.txt"
typst_file = typst_path / "code.txt"
typst_file = typst_path / "color_eq.txt"

# Manual SVG conversion + add to scene

In [ ]:
from typst_importer.svg_preprocessing import preprocess_svg, stroke_to_filled_path

typst.compile(typst_file, format="svg", output=str(svg_file))

step1_content = svg_file.read_text()
step2_content = preprocess_svg(step1_content)
step3_content = stroke_to_filled_path(step2_content)


display_svg(step1_content , width='500px')
display_svg(step3_content , width='500px')

[I 2025-02-07 11:37:21.838 ServerApp] 302 GET / (@127.0.0.1) 0.87ms
[I 2025-02-07 11:37:21.840 LabApp] 302 GET /lab? (@127.0.0.1) 0.92ms
[I 2025-02-07 11:37:21.875 ServerApp] Kernel started: 390e105b-0307-45d7-9f0c-dfb5e58c8be6


# SVG Blender Operations

In [397]:
output_file = temp_dir / "step3.svg"
output_file.write_text(step3_content)

bpy.ops.import_curve.svg(filepath=str(output_file))
col = bpy.context.scene.collection.children['step3.svg']
col.name = "Formula"

for obj in col.objects:
    obj.scale = (100, 100, 100) # this is the old method, #better 
    # obj.data.transform(Matrix.Scale(scale_factor, 4))

In [398]:
# Loop through each object in the collection and set its origin
bpy.ops.object.select_all(action='DESELECT')
if col.objects:
    # Set the first object as active
    bpy.context.view_layer.objects.active = col.objects[0]
    # Now we can safely set the mode to OBJECT
    bpy.ops.object.mode_set(mode='OBJECT') 
    for obj in col.objects:
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='MEDIAN')
        obj.select_set(False)

# Function overview


In [125]:
import tempfile
from pathlib import Path
from typst_importer.typst_to_svg import typst_to_blender_curves
typst_file =  Path(tempfile.gettempdir()) / "hello.typ"

header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a= b/c $"
typst_file.write_text(header+body)
collection = typst_to_blender_curves(typst_file)

In [127]:
from typst_importer.typst_to_svg import typst_express

c = typst_express("$ . . . $", scale_factor=100, origin_to_char=False, convert_to_mesh=True)
print(c.processed_svg)

<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 41.7 5.3" width="41.7pt" height="5.3pt">
    <g>
        <g transform="translate(0 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(13.900000000000002 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(27.800000000000004 5.3)">
            <g class="typst-text" transform="scale(1, -1

In [130]:
svg_content = """
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:h5="http://www.w3.org/1999/xhtml" class="typst-doc" viewBox="0 0 41.7 5.3" width="41.7pt" height="5.3pt">
    <g>
        <g transform="translate(0 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(10 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#FFFFFF" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(20 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#000000" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
        <g transform="translate(30 5.3)">
            <g class="typst-text" transform="scale(1, -1)">
                <g fill="#FFFFFF" fill-rule="nonzero"><path d="M 9.6 2.65 C 9.6 4.1 8.400001 5.3 6.9500003 5.3 C 5.5 5.3 4.3 4.1 4.3 2.65 C 4.3 1.2 5.5 0 6.9500003 0 C 8.400001 0 9.6 1.2 9.6 2.65 Z "/>
        </g></g>
        </g>
    </g>
</svg>
"""
from pathlib import Path
import tempfile
import bpy

def deduplicate_materials(collection):
    """
    Deduplicate materials in a collection by reusing identical materials and giving them descriptive names.
    This reduces memory usage by ensuring identical materials are shared across objects.
    
    Args:
        collection: bpy.types.Collection - The collection containing objects whose materials need deduplication
    """
    materials_dict = {}  # Dictionary to store unique materials
    
    for obj in collection.objects:
        if obj.data.materials:
            current_mat = obj.data.materials[0]
            mat_key = (
                tuple(current_mat.diffuse_color),
                current_mat.metallic,
                current_mat.roughness
            )
            
            if mat_key in materials_dict:
                # If we've seen this material before, use the existing one
                obj.data.materials.clear()
                obj.data.materials.append(materials_dict[mat_key])
            else:
                # If this is a new material, add it to our dictionary
                rgb = current_mat.diffuse_color[:3]  # Get RGB values (ignore alpha)
                hex_color = ''.join(f'{int(c*255):02x}' for c in rgb)
                current_mat.name = f"Mat{len(materials_dict)}_#{hex_color}"
                materials_dict[mat_key] = current_mat

temp_dir = Path(tempfile.gettempdir())
svg_file = temp_dir / "my_example.svg"
svg_file.write_text(svg_content)
# Delete all objects
bpy.ops.object.select_all(action='SELECT')
bpy.ops.object.delete()

# Delete all collections except the Scene Collection
for collection in bpy.data.collections:
    bpy.data.collections.remove(collection)

bpy.ops.outliner.orphans_purge() # remove unused materials

bpy.ops.import_curve.svg(filepath=str(svg_file))
imported_collection = bpy.context.scene.collection.children.get(svg_file.name)
imported_collection.name = f"Hello"

# Handle materials
deduplicate_materials(imported_collection)
bpy.ops.outliner.orphans_purge() # remove unused materials
# Convert curves to meshes
for obj in imported_collection.objects:
    if obj.type == 'CURVE':
        # Store reference to original curve data
        curve_data = obj.data
        original_name = obj.name.replace('Curve', '')
        
        # Select the curve object
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        
        # Convert to mesh
        bpy.ops.object.convert(target='MESH')
        
        # Rename both the object and its mesh data
        new_name = f"Mesh{original_name}"
        obj.name = new_name
        obj.data.name = new_name
        
        obj.select_set(False)
        
        # Now that the object has been converted, we can remove the original curve data
        bpy.data.curves.remove(curve_data)

Info: Deleted 6 data-block(s)
Info: Deleted 3 data-block(s)


Info: Saved "untitled.blend"
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full (Size: 64), vertex descriptor will not be cached
[Performance Warning] cache is full 